# Tabalho de I.A - UFRJ - 2019.1

Bruno Hryniewicz dos Santos Cruz  - DRE - 115150099

# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import numpy as np
import scipy.linalg as linalg
import math

# Iris Plants Database 
(https://archive.ics.uci.edu/ml/machine-learning-databases/iris/)

## Importando o Dataframe

In [2]:
df =pd.read_csv("irisDataset.txt",  names = ["1", "2", "3", "4", "Class"])

## Separando modelo de treino e teste

In [3]:
train, test = train_test_split(df, test_size=0.2)

In [4]:
print("Treino tem", train.shape[0], "linhas")
print("Teste tem", test.shape[0], "linhas")

Treino tem 120 linhas
Teste tem 30 linhas


## Precepton
Como é um problema de 3 classes primeiro irei ajustar a reta entre ser da classe 1 ou não, depois quem não for da classe 1, irei ajustar a reta que divide entre classe 2 e 3.

### Inicialização de Array de pesos

In [5]:
# definindo pesos iniciais para primeira rodada do precepton
w1 = np.array([random.randint(1,100),random.randint(1,100),random.randint(1,100),random.randint(1,100)], dtype = float)


# definindo pesos iniciais para segunda rodada do precepton
w2 = np.array([random.randint(1,100),random.randint(1,100),random.randint(1,100),random.randint(1,100)], dtype = float)


### Preditor
Definindo o funcionamento do preditor antes de ajustar as retas para que possa usar ele no criterio de parada na segunda reta (visto que ja foi verificado que não é linermente separavel).

In [6]:
def preditor(entrada, w1, w2):
    h1 = 0
    for j in range(4):
        h1 += entrada[j] * w1[j]
    h1 = h1- 100
    if h1 > 0:
        return "Iris-setosa"
    else:
        h = 0
        for j in range(4):
            h += entrada[j] * w2[j]
        h = h- 100
        if h > 0 :
            return "Iris-virginica"
        else: 
            return "Iris-versicolor"

### Teste
Para cada valor no meu conjunto de teste verifico a resposta do preditor, se for igual ao esperado boto 1 no vetor respostas, se não boto 0, a minha precisão será a media desse vetor.

In [7]:
def teste(w1, w2, test_set):
    respostas = []
    for item in test_set:
        if preditor(item[0:4],w1,w2) == item[4]:
            respostas.append(1)
        else:
            respostas.append(0)
    respostas = np.array(respostas) 
    return respostas.mean()

### Classificando entre Iris-setosa ou não Iris-setosa

Conjuntos lineramente separaveis, pois o precepton termina.


In [8]:
# definindo pesos iniciais para primeira rodada do precepton
w_temp1 = np.array([random.randint(1,100),random.randint(1,100),random.randint(1,100),random.randint(1,100)], dtype = float)


def fase1 (w1, itens):
    print("pesos iniciais fase 1 = ", w1)
    i=0
    flag = True
    while (i < len(itens)):
        h = 0
        item = itens[i]
        classe = 0
        predict = 0

        # definindo classe com 1 se é da classe alvo e -1 se não é
        if item[4] == "Iris-setosa":
            classe = 1
        else:
            classe = -1

        # fazendo somatorido de Xn*Wn
        for j in range(4):
            h += item[j] * w1[j]

        # descontando threshwolder arbitrariamente definido como 100
        h = h - 100

        # retirando a classe dada pela predição
        if h > 0:
            predict = 1
        else:
            predict = -1

        # Ajustando vetor de pesos
        # se a classe for igual a predição classe - predict = 0 e não modifica o vetor
        # se não classe - predict = +-1 dependendo da direção do erro.
        w1 = w1 + (classe - predict) * item[0:4]

        # reiniciando loop caso tenha ajustado os pesos
        if (classe - predict) != 0:
            i = 0 
            continue
        # incremento de loop
        i+= 1
    print("pesos finais fase 1 = ", w1)
    return w1
w_fase1 = fase1(w_temp1, train.values)


pesos iniciais fase 1 =  [50. 77. 41. 95.]
pesos finais fase 1 =  [5.329070518200751e-15 69.19999999999999 -54.599999999999994
 59.59999999999996]


### Classificando quem não é Iris-setosa em Iris-virginica ou Iris-versicolor

Aparetemente os conjuntos não são lineramente independentes, primeiro utilizei da estatégia de parada mais simpres possivel (limitei o numero de iterações) poré os resultados variavam de treino para treino. 

Então resolvi elaborar mais minha estratégia de parada, dessa vez guardando o melhor resultado. Assim, cheguei a precisão de quase 97%.

Porem rodando mais algumas vezes, percebi que essa precisão se altera dependendo do conjunto de treino e teste que seleciono aleatoriamente.

Então caso não chege numa precisão boa o suficiente tente rodar novamento o split dos testes e então treinar novamente.

Outo fator determinante é que como começo com pesos aleatóios a cada vez que treino, como defino uma quantidade de iteraçoes até a parada, terei modelos diferente a cada treinamento no mesmo conjunto, visto que os ajustes de retas diferentes levam tempos diferentes.


In [9]:
# definindo pesos iniciais para segunda rodada do precepton
w_temp2 = np.array([random.randint(1,100),random.randint(1,100),random.randint(1,100),random.randint(1,100)], dtype = float)

def fase2(w1, w2, itens, test_set):
    print("pesos iniciais fase 2 = ", w2)
    i=0
    flag = True
    stop =0
    maior_teste = 0
    melhor_w = []

    while (i < len(itens)):
        h = 0
        item = itens[i]
        classe = 0
        predict = 0
        h1 = 0
        # Verificando se o objeto pretence a classe Iris-setosa que ja foi classificada
        # Se pertence pula para o próximo
        for j in range(4):
            h1 += item[j] * w1[j]
        h1 = h1- 100

        if ( h1 > 0):
            i+=1
            continue

        # definindo classe com 1 se é da classe alvo e -1 se não é    
        if item[4] == "Iris-virginica":
            classe = 1
        else:
            classe = -1
            
        # fazendo somatorido de Xn*Wn
        for j in range(4):
            h += item[j] * w2[j]
        # descontando threshwolder arbitrariamente definido como 100
        h = h - 100

        # retirando a classe dada pela predição
        if h > 0:
            predict = 1
        else:
            predict = -1

        # Ajustando vetor de pesos
        # se a classe for igual a predição classe - predict = 0 e não modifica o vetor
        # se não classe - predict = +-1 dependendo da direção do erro.
        w2 = w2 + (classe - predict) * item[0:4]

        # reiniciando loop caso tenha ajustado os pesos
        if (classe - predict) != 0:
            i = 0 
            continue
        # verificando a precisão da predição atual
        t = teste(w1, w2, test_set )
        # trocando melhor prediçao caso seja mais precisa
        if t > maior_teste:
            maior_teste = t
            melhor_w = w2
        # incementando condição de parada para conjunto não lineramente separavel     
        stop += 1

        #condição de parada para conjunto não lineramente separavel
        if stop > 10000:
            break

        # incremento de loop
        i+= 1
        #redefinindo melhor predição
    print("pesos finais fase 2 = ", melhor_w)
    return melhor_w

w_fase2 = fase2(w_fase1,w_temp2,train.values, test.values)


pesos iniciais fase 2 =  [91. 58.  9. 87.]
pesos finais fase 2 =  [-55.99999999999934 84.59999999999492 -108.20000000000415
 423.80000000001246]


In [10]:
print ("Minha precisão é de:",teste(w_fase1, w_fase2, test.values ) *100 , "%")

Minha precisão é de: 86.66666666666667 %


### Fixando as aleatoriedades do modelo para ter um modelo treinado.


In [11]:
#salvando os conjuntos de teste e de treino
# train.to_csv("iris_traing.csv")
# test.to_csv("iris_test.csv")

In [12]:
# inicializando os vetores de pesos com pesos onde em teste anteriores deram um bom modelo preditivo
# o peso w1 sempre dará um bom modelo preditivo pois ele separa dosi conjuntos lineramente separaveis
# ja o peso 2 como foi explicado anteriormente, seu valor influencia na precisão
w1_inicial_modelo = np.array([52., 35., 55., 29.], dtype = float)
w2_inicial_modelo = np.array([ 71.,  27.,  37., 100.], dtype = float)

# carregando conjuntos de treino e teste

treino_modelo = pd.read_csv("iris_traing.csv", index_col = 0)
teste_modelo = pd.read_csv("iris_test.csv", index_col = 0)

w1_modelo = fase1(w1_inicial_modelo, treino_modelo.values)

w2_modelo = fase2(w1_modelo, w2_inicial_modelo, treino_modelo.values, teste_modelo.values)
print ("Minha precisão é de:",teste(w1_modelo, w2_modelo, teste_modelo.values ) *100 , "%")



pesos iniciais fase 1 =  [52. 35. 55. 29.]
pesos finais fase 1 =  [15.00000000000001 45.800000000000004 -35.19999999999999 4.000000000000011]
pesos iniciais fase 2 =  [ 71.  27.  37. 100.]
pesos finais fase 2 =  [-87.00000000000034 -37.59999999999994 98.7999999999999 175.59999999999883]
Minha precisão é de: 100.0 %


# Discrimination in Salaries 
(http://data.princeton.edu/wws509/datasets/#salary)

In [13]:
texto = []
f = open("DiscriminationInSalariesDataset.txt" )
for line in f:    
    texto.append(line.split())
f.close()

In [14]:
df2 = pd.DataFrame(texto[1:], columns = texto[0] )

In [15]:
# train2_randon, test2_randon = train_test_split(df2, test_size=0.2)
# train2_randon.to_csv("salaries_traing.csv")
# test2_randon.to_csv("salaries_test.csv")

train2 = pd.read_csv("salaries_traing.csv", index_col = 0)
test2 = pd.read_csv("salaries_test.csv", index_col = 0)

## Divisão de conjunto de teste e de Treino

In [16]:
print("Treino tem", train2.shape[0], "linhas")
print("Teste tem", test2.shape[0], "linhas")

Treino tem 41 linhas
Teste tem 11 linhas


## traduzindo stiirngs para valores

In [17]:
def traduz_sx(x):
    if x == "male": 
        return 1
    else:
        return 2
    
def traduz_rk(x):
    if(x =="full"):
        return 1
    elif (x == "associate"):
        return 2
    else:
        return 3

def traduz_dg(x):
    if (x == "doctorate"):
        return 1
    else:
        return 2


## Regressão linear
Utilizarei a regressão linear para tentar prever os slários.

### Considerando sexo como valor relevante na predição

In [18]:
X = []
y = []

for pessoa in train2.values:
    aux = np.transpose([traduz_sx(pessoa[0]),traduz_rk(pessoa[1]), int(pessoa[2]), traduz_dg(pessoa[3]), int(pessoa[4])])
    X.append(aux)
    y.append(int(pessoa[5]))

pseudo_inv = linalg.pinv(np.array(X))    
linear_regression= pseudo_inv.dot(np.transpose(np.array([y])))




In [19]:
def predict(x):
    aux = [traduz_sx(x[0]),traduz_rk(x[1]), int(x[2]), traduz_dg(x[3]), int(x[4])]
    resp = 0
    for i in range(5):
        resp += aux[i] * linear_regression[i][0]
    return resp

In [20]:
erro = 0
for i in test2.values:
    print("Salário real:" , i[5], "Salário predito:", predict(i))
    aux = predict(i) - int(i[5])
    erro += (aux) ** 2
    
print ( "Erro quadratico médio: ", erro/len(test2.values))
print ( "Raiz do erro quadratico médio:", math.sqrt(erro/len(test2.values)))

Salário real: 24742 Salário predito: 26661.620092620637
Salário real: 25748 Salário predito: 21486.647093123305
Salário real: 25400 Salário predito: 17516.92040114977
Salário real: 35350 Salário predito: 24994.71719150052
Salário real: 28200 Salário predito: 21082.697573726527
Salário real: 25500 Salário predito: 25410.042391733805
Salário real: 15000 Salário predito: 19175.962280179505
Salário real: 31850 Salário predito: 30269.17027909387
Salário real: 20690 Salário predito: 29425.49320953648
Salário real: 16500 Salário predito: 12188.711351289547
Salário real: 24832 Salário predito: 19921.667326836377
Erro quadratico médio:  34620734.91459581
Raiz do erro quadratico médio: 5883.938724578615


### Desconsiderando o sexo no preditor

In [21]:
X = []
y = []

for pessoa in train2.values:
    aux = np.transpose([traduz_rk(pessoa[1]), int(pessoa[2]), traduz_dg(pessoa[3]), int(pessoa[4])])
    X.append(aux)
    y.append(int(pessoa[5]))

pseudo_inv = linalg.pinv(np.array(X))    
linear_regression= pseudo_inv.dot(np.transpose(np.array([y])))

In [22]:
def predict(x):
    aux = [traduz_rk(x[1]), int(x[2]), traduz_dg(x[3]), int(x[4])]
    resp = 0
    for i in range(4):
        resp += aux[i] * linear_regression[i][0]
    return resp

In [23]:
erro = 0
for i in test2.values:
    print("Salário real:" , i[5], "Salário predito:", predict(i))
    aux = predict(i) - int(i[5])
    erro += (aux) ** 2
    
print ( "Erro quadratico médio: ", erro/len(test2.values))
print ( "Raiz do erro quadratico médio:", math.sqrt(erro/len(test2.values)))

Salário real: 24742 Salário predito: 24405.386535656275
Salário real: 25748 Salário predito: 23057.58537112007
Salário real: 25400 Salário predito: 17619.418041420366
Salário real: 35350 Salário predito: 24344.947124782535
Salário real: 28200 Salário predito: 19425.630492087486
Salário real: 25500 Salário predito: 16581.716647411056
Salário real: 15000 Salário predito: 13682.784458584623
Salário real: 31850 Salário predito: 30265.083554609057
Salário real: 20690 Salário predito: 30194.689270319108
Salário real: 16500 Salário predito: 15301.751639511875
Salário real: 24832 Salário predito: 20435.822714021793
Erro quadratico médio:  41897630.09350227
Raiz do erro quadratico médio: 6472.837870169642
